# Plotting the position of the mouse over time and generating a video


To do that, I must: 

- Calibrate the extrinsics of the fm cameras in reference to the calibrador. It will probably be slightly off focus, because the headstage is not in the same position, so I should calculate the offset. 
- Figure out how to plot points on that space on the camera
- Figure out how to translate the points of the lighthouse positioning into the camera frame (should be simple, just offset)
- Figure out how to make a video

### Imports

In [ ]:
from pathlib import Path
import cv2
import pandas as pd
import flexiznam as flz
import numpy as np
import cottage_analysis.io_module.onix as onix
import cottage_analysis.io_module.harp as harp
import cottage_analysis.ephys.preprocessing as prp
import cottage_analysis.utilities.plot_utils as plut
from matplotlib import pyplot as plt
import math
import znamcalib.calibrate_lighthouse as light

### Calibrating the extrinsics

Done. The intrinsic calibration of the cameras used is S20220627, and the extrinsic is S20230509

### Dealing with OpenCV: project 3d point to frame

The function is documented as follows: 

In [ ]:
"""
Python: cv.ProjectPoints2(objectPoints, rvec, tvec, cameraMatrix, distCoeffs, imagePoints, dpdrot=None, dpdt=None, dpdf=None, dpdc=None, 
    dpddist=None) → None

Parameters:	
- objectPoints – Array of object points, 3xN/Nx3 1-channel or 1xN/Nx1 3-channel (or vector<Point3f> ), where N is the number of points in the view.
- rvec – Rotation vector. See Rodrigues() for details.
- tvec – Translation vector.
- cameraMatrix – Camera matrix A = \vecthreethree{f_x}{0}{c_x}{0}{f_y}{c_y}{0}{0}{_1} .
- distCoeffs – Input vector of distortion coefficients (k_1, k_2, p_1, p_2[, k_3[, k_4, k_5, k_6]]) of 4, 5, or 8 elements. If the vector is 
    NULL/empty, the zero distortion coefficients are assumed.
- imagePoints – Output array of image points, 2xN/Nx2 1-channel or 1xN/Nx1 2-channel, or vector<Point2f> .
- jacobian – Optional output 2Nx(10+<numDistCoeffs>) jacobian matrix of derivatives of image points with respect to components of the 
    rotation vector, translation vector, focal lengths, coordinates of the principal point and the distortion coefficients. In the old 
    interface different components of the jacobian are returned via different output parameters.
- aspectRatio – Optional “fixed aspect ratio” parameter. If the parameter is not 0, the function assumes that the aspect ratio (fx/fy) is fixed 
    and correspondingly adjusts the jacobian matrix.


The function computes projections of 3D points to the image plane given intrinsic and extrinsic camera parameters. Optionally, the function 
computes Jacobians - matrices of partial derivatives of image points coordinates (as functions of all the input parameters) with respect to the 
particular parameters, intrinsic and/or extrinsic. The Jacobians are used during the global optimization in calibrateCamera(), solvePnP(), and 
stereoCalibrate() . The function itself can also be used to compute a re-projection error given the current intrinsic and extrinsic parameters.

"""


So, we must get for each (static) plot: 

- A set of position points (from the Lighthouse tracking)
- The rvec and tvec of the extrinsic calibration
- The cameraMatrix and distCoeffs
- what does imagePoints mean?



#### Loading data

In [ ]:
#This is all done for cam2_camera only

DATA_PATH = Path('/camp/lab/znamenskiyp/data/instruments/raw_data/projects/blota_onix_pilote')
PROCESSED_PATH = Path('/camp/lab/znamenskiyp/home/shared/projects/blota_onix_pilote')
INTRINSICS_PATH = Path('/camp/lab/znamenskiyp/home/shared/projects/blota_onix_calibration/camera_intrinsics')
EXTRINSICS_PATH = Path('/camp/lab/znamenskiyp/home/shared/projects/blota_onix_calibration/arena_extrinsics')
INTRINSICS_SESSION = 'S20220627'
EXTRINSICS_SESSION = 'S20230509'

MOUSE = 'BRAC7448.2d'
SESSION = 'S20230412'
CAMERA = 'cam2_camera'

#Temporary and painful

ephys = 'R163257'
camera = 'R162624_freelymoving'

In [ ]:
#Formulating complete paths

data_path = DATA_PATH / MOUSE / SESSION / ephys
camera_path = DATA_PATH / MOUSE / SESSION / camera
processed_path = PROCESSED_PATH / MOUSE / SESSION
intrinsics_path = INTRINSICS_PATH / INTRINSICS_SESSION / CAMERA
extrinsics_path = EXTRINSICS_PATH / EXTRINSICS_SESSION / CAMERA / 'cam2_camera_snapshot_0_extrinsics_0'

In [ ]:
#Loading Lighthouse data
processed_photodiode = onix.load_ts4231(data_path)
diode3 = processed_photodiode[3]
calibration = Path('/camp/lab/znamenskiyp/data/instruments/raw_data/projects/blota_onix_calibration/lighthouse_calibration/S20230412')
transform_matrix = light.calibrate_session(calibration, 20)
trans_data = light.transform_data(diode3, transform_matrix)
light.plot_single_occupancy(trans_data)

In [ ]:
#Loading extrinsics data

#Will create a function that loads the .npz files for each marker. 

def load_extrinsics(extrinsics_path):
    """
    Inside of the .npz files you find a dictionary with the following keys: 
    ['rvec', 'tvec', 'corners', 'r2m_rvec', 'r2m_tvec', 'm2r_rvec', 'm2r_tvec']

    """
    marker_files = list(extrinsics_path.glob('*.npz'))
    out = dict()
    for i, marker in enumerate(marker_files): 
        marker_path = extrinsics_path / marker
        out[f'{str(marker.name)[0:7]}'] = np.load(marker_path)
    return(out)

extrinsics = load_extrinsics(extrinsics_path)


In [ ]:
rvec = extrinsics['marker4']['rvec']
tvec = extrinsics['marker4']['tvec']

In [ ]:
#Loading intrinsics data

def load_intrinsics(intrinsics_path):
    """
    outputs a dictionary with the following set of keys: 
    ['ret', 'mtx', 'dist', 'rvecs', 'tvecs', 'mean_error']
    where mtx is the cameraMatrix and dist are the distCoeffs
    
    """
    files = list(intrinsics_path.glob('*.npz'))
    intrinsics = np.load(files[0])
    return(intrinsics)

intrinsics = load_intrinsics(intrinsics_path)

In [ ]:
cameraMatrix = intrinsics['mtx']
distCoeffs = intrinsics['dist']

We now have to load a frame of the video at a specific onix time, so that we are able to plot the position of the mouse on that frame. To do that, loading metadata of the cameras, setting the video and grabbing the frame. 

In [ ]:
#Fuck, Antonin changed the structure of the cameras!

camera_dir = camera_path
acquisition= 'freely_moving'

def load_camera_times(camera_dir, acquisition):
    if acquisition == "freely_moving":
        camlist = ["cam1_camera", 
                   "cam2_camera", 
                   "cam3_camera"]
    if acquisition == "headfixed":
        camlist = [
            "letfeye_camera",
            "righteye_camera",
            "face_camera",
            "behaviour_camera",
        ]
    folder = Path(camera_dir)
    if not folder.is_dir():
        raise IOError("%s is not a directory" % folder)
    output = dict()
    for cam in camlist:
        valid_files = list(folder.glob(f"{cam}*timestamps*"))
        if not len(valid_files):
            raise IOError(f"Could not find any timestamp files in {folder}")
        for possible_file in valid_files:
            possible_file = str(possible_file)
            if cam in possible_file:
                output[cam] = pd.read_csv(possible_file)
    return output

camera_metadata = load_camera_times(camera_dir, acquisition)
print(camera_metadata.keys())

In [ ]:
#Translating camera clock into onix clock. This is tthe dirty way, but should probably work. Now, we've got the HARP. 

processed_breakout = onix.load_breakout(data_path)
dio = processed_breakout['dio']

filtered_dio = dict()
filtered_dio['Clock'], filtered_dio['DI0'] = prp.clean_di_channel(dio['Clock'], dio['DI0'])

cam2_metadata = camera_metadata['cam2_camera']
camera_frames = len(cam2_metadata['frame_id'])
breakout_frames = (len(filtered_dio['DI0']))/2

print(f'cam2 has {camera_frames} frames')
print(f'The breakout board accounts for {breakout_frames} frames')

def cam2onix(filtered_dio, cam2_metadata):

    onix_first = filtered_dio['Clock'][filtered_dio['DI0']==True][0]
    onix_last = filtered_dio['Clock'][filtered_dio['DI0']==True][-1]
    camera_first = cam2_metadata['camera_timestamp'].iloc[0]
    camera_last = cam2_metadata['camera_timestamp'].iloc[-1] 

    #y=intercept+slope*x

    intercept = onix_first
    slope = (onix_last-onix_first)/(camera_last-camera_first)

    onix_timestamp = intercept+slope*(cam2_metadata['camera_timestamp']-camera_first)

    return onix_timestamp

cam2_metadata['onix_timestamp'] = cam2onix(filtered_dio, cam2_metadata)
cam2_metadata['onix_timestamp']

plt.plot(filtered_dio['Clock'][filtered_dio['DI0']==True], color = 'blue')
plt.plot(cam2_metadata['onix_timestamp'], color = 'red')

In [ ]:
# Finding a frame at a specific onix time

def find_nearest(Array, x):
    dif_Array = np.absolute(Array-x) # use of absolute() function to find the difference 
    index = dif_Array.argmin() # find the index of minimum difference element
    return Array[index]

onix_time = trans_data['clock'][20000]

video_path = '/camp/lab/znamenskiyp/data/instruments/raw_data/projects/blota_onix_pilote/BRAC7448.2d/S20230412/R162624_freelymoving/cam2_camera_2023-04-12T16_26_24.mp4'


#We want to normalise luminance to the first frame

def get_avg_first_frame(video_path):
    video = cv2.VideoCapture(video_path)
    video.set(cv2.CAP_PROP_POS_FRAMES, 1)
    ret, frame_start = video.read()
    frame_start = cv2.cvtColor(frame_start, cv2.COLOR_BGR2GRAY)
    avg_first_frame = frame_start.mean()
    return(avg_first_frame)


def get_onix_frame(video_path, onix_time, avg_first_frame=None, show=False, normalise = False):
    start_frame_onix = find_nearest(cam2_metadata['onix_timestamp'], onix_time)
    start_frame = cam2_metadata[cam2_metadata['onix_timestamp']==start_frame_onix]
    start_frame = start_frame.index[0]
    video = cv2.VideoCapture(video_path)
    video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    ret, frame_start = video.read()
    if normalise==True:
        frame_start = frame_start*(avg_first_frame / frame_start.mean())
    if show==True:
        fig, axs = plt.subplots(1, 1, figsize=(50, 50))
        fig.suptitle(f'Frame {start_frame}', size = 40)
        axs.imshow(frame_start)
    return frame_start

frame_start = get_onix_frame(video_path, onix_time)


In [ ]:
avg_first_frame = get_avg_first_frame(video_path)


### Running the transformation

In the current calibration, marker 4 is the reference. By looking at the pictures, z points upwards in the arena, x points left in the arena, y points downwards in the arena, from the reference point of an observer opening the door and looking inside. I will use marker4 as the source of calibration data. 

As far as I've seen: rvec and tvec are the translation and rotation vectors that show the position of the marker in the frame of reference of the camera. That is: z points form the camera forwards, x points from the center of the camera frame to the right, y points upwards. The metadata from the extrinsics calibration show that the calibration was done in mm, and the 0,0 of the lihgthouse calibration is 4cm further along the y axis than the centre of the Aruco. Thus, I'll convert the LIghthouse coordinates to mm and substract 6cm from each point. 

In [ ]:
video_path = '/camp/lab/znamenskiyp/data/instruments/raw_data/projects/blota_onix_pilote/BRAC7448.2d/S20230412/R162624_freelymoving/cam2_camera_2023-04-12T16_26_24.mp4'

def lighthouse2aruco(trans_data):
    #transforming coordinates
    proj_data = trans_data.copy(deep=True)
    proj_data['x'] = (trans_data['x']*10)
    proj_data['y'] = ((trans_data['y'])*10)+(44.5) #25 for half an aruco, 25 for the other half, 15 until the connector, 4.5 for one connector and a half
    proj_data['z'] = trans_data['z']*10 
    return proj_data

proj_data = lighthouse2aruco(trans_data)

def _plot_onix_time(onix_time, video_path, proj_data, rvec, tvec, cameraMatrix, distCoeffs):
    """
    Same as plot_onix_time, but for efficiency I took out the transformation step to use it in generating the videos, so it works directly with
    proj_data. 
    """

    #obtaining the closest frame
    frame_start = get_onix_frame(video_path, onix_time)
    #selecting index
    lighthouse_index = find_nearest(proj_data['clock'], onix_time)
    index = proj_data[proj_data['clock']==lighthouse_index].index[0]
    objectPoints = np.array([proj_data['x'][index], proj_data['y'][index], proj_data['z'][index]])
    objectPoints = np.transpose(objectPoints)
    #transforming and plotting
    imagePoints, jacobian = cv2.projectPoints(objectPoints, rvec, tvec, cameraMatrix, distCoeffs)
    coordinates = imagePoints[0][0]
    fig, axs = plt.subplots(1, 1, figsize=(50, 50))
    axs.imshow(frame_start)
    axs.scatter(coordinates[0], coordinates[1], marker='+', s=15e3)
    return fig

def plot_onix_time(onix_time, video_path, trans_data, rvec, tvec, cameraMatrix, distCoeffs):
    #obtaining the closest frame
    frame_start = get_onix_frame(video_path, onix_time)
    #transforming coordinates
    proj_data = lighthouse2aruco(trans_data)
    #selecting index
    lighthouse_index = find_nearest(proj_data['clock'], onix_time)
    index = proj_data[proj_data['clock']==lighthouse_index].index[0]
    objectPoints = np.array([proj_data['x'][index], proj_data['y'][index], proj_data['z'][index]])
    objectPoints = np.transpose(objectPoints)
    #transforming and plotting
    imagePoints, jacobian = cv2.projectPoints(objectPoints, rvec, tvec, cameraMatrix, distCoeffs)
    coordinates = imagePoints[0][0]
    fig, axs = plt.subplots(1, 1, figsize=(50, 50))
    axs.imshow(frame_start)
    axs.scatter(coordinates[0], coordinates[1], marker='+', s=15e3)
    return fig
    

time_in_s = 1400

onix_time = trans_data['clock'][0]+(time_in_s*250e6)

#plot_onix_time(onix_time, video_path, trans_data, rvec, tvec, cameraMatrix, distCoeffs)
    
    
    


### Creating a video

In [ ]:
def plot_interval_s(start, end, video_path, trans_data, rvec, tvec, cameraMatrix, distCoeffs):

    frame_times_s = np.arange(start, end, (1/15))

    # Define the output file name, codec, frame rate, and dimensions
    output_file = f'output_video_{str(start)}_{str(end)}.mp4'
    codec = cv2.VideoWriter_fourcc(*'mp4v')  # Choose the appropriate codec
    frame_rate = 15  # Desired frame rate
    frame_width = 3600 #as measured from current matplotlib output, maybe better to change programatically to ensure robustness when changing
    #plotting function?
    frame_height = 3600

    # Create the video writer
    video_writer = cv2.VideoWriter(output_file, codec, frame_rate, (frame_width, frame_height))

    proj_data = lighthouse2aruco(trans_data)

    for i in frame_times_s:
        time_in_s = i
        onix_time = proj_data['clock'][0]+(time_in_s*250e6)
        fig = _plot_onix_time(onix_time, video_path, proj_data, rvec, tvec, cameraMatrix, distCoeffs)
        image = plut.get_img_from_fig(fig)
        plt.close(fig)
        bgr_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # Write the frame to the video
        video_writer.write(bgr_image)

    video_writer.release()
    cv2.destroyAllWindows()

start = 1
end = 20

#plot_interval_s(start, end, video_path, trans_data, rvec, tvec, cameraMatrix, distCoeffs)

In [ ]:
duration = proj_data['clock'][len(proj_data['clock'])-1]-proj_data['clock'][0]
duration_s = duration/250e6
duration_s

In [ ]:
#plotting a diagonal

diagonal = np.arange(0, 70, 0.5)
z = np.arange(0,40,(40/140))
zeros = np.zeros(140)
seventies = np.array([70]*140)
diagonal_points = np.zeros((140, 3))
diagonal_points[:,0]=(seventies*10)-25
diagonal_points[:,1]=(seventies*10)-25
diagonal_points[:,2]=z*10

In [ ]:
proj_data = lighthouse2aruco(trans_data)
whole_traj = np.zeros((len(proj_data), 3))
whole_traj[:,0] = pd.array(proj_data['x'])
whole_traj[:,1] = pd.array(proj_data['y'])
whole_traj[:,2] = pd.array(proj_data['z'])

In [ ]:
#plotting a grid
grid_points = np.zeros((200, 3))
o = -1
for i in np.arange(0, 70, 5):
    for e in np.arange(0, 70, 5):
        o = o+1
        grid_points[o, 0] = (i*10)-25
        grid_points[o, 1] = (e*10)-25
grid_points


In [ ]:
def plot_point_series(series, frame_start, rvec, tvec, cameraMatrix, distCoeffs):
    fig, axs = plt.subplots(1, 1, figsize=(50, 50))
    axs.imshow(frame_start)
    for i in series[:,]:
        objectPoints = np.transpose(i)
        #transforming and plotting
        imagePoints, jacobian = cv2.projectPoints(objectPoints, rvec, tvec, cameraMatrix, distCoeffs)
        coordinates = imagePoints[0][0]
        axs.scatter(coordinates[0], coordinates[1], marker='.', s=90)
    return fig

#plot_point_series(grid_points, frame_start, rvec, tvec, cameraMatrix, distCoeffs)


### Trying with warping the image to make it square. 

Alternatively, we can plot 

### Plotting orientation

Loading orientation

In [ ]:
processed_bno = onix.load_bno055(data_path)
processed_bno.keys()

quaternion = processed_bno['quaternion']
#quaternion = quaternion/2**14 but the rest make sense (???)
quaternion = quaternion
linear = processed_bno['linear']
linear = linear 
euler = processed_bno['euler']
euler = euler
gravity = processed_bno['gravity']
gravity = gravity 

In [ ]:
def get_heading(processed_bno):
    quaternion = processed_bno['quaternion']
    Qw = np.transpose(quaternion[:,0])
    Qx = np.transpose(quaternion[:,1])
    Qy = np.transpose(quaternion[:,2])
    Qz = np.transpose(quaternion[:,3])
    heading = np.zeros(len(Qw))
    for i in np.arange(0, len(Qw)):
        ith_quaternion = [[Qw[i], Qx[i], Qy[i], Qz[i]]]
        norm_quaternion = ith_quaternion / np.linalg.norm([Qw[i], Qx[i], Qy[i], Qz[i]])
        norm_quaternion = norm_quaternion[0]
        heading[i] = _get_heading(norm_quaternion[0], norm_quaternion[1],norm_quaternion[2],norm_quaternion[3])
    return heading

In [ ]:
#The quaternion is supposed to output stuff in the order w, x, y, z
# It's an axes*values matrix

Qw = np.transpose(quaternion[:,0])
Qx = np.transpose(quaternion[:,1])
Qy = np.transpose(quaternion[:,2])
Qz = np.transpose(quaternion[:,3])

def _get_heading(Qw, Qx, Qy, Qz):
    heading = math.atan2(2.0 * (Qz * Qw + Qx * Qy) , - 1.0 + 2.0 * (Qw * Qw + Qx * Qx))
    return heading


heading = np.zeros(len(Qw))

for i in np.arange(0, len(Qw)):
    ith_quaternion = [[Qw[i], Qx[i], Qy[i], Qz[i]]]
    norm_quaternion = ith_quaternion / np.linalg.norm([Qw[i], Qx[i], Qy[i], Qz[i]])
    norm_quaternion = norm_quaternion[0]
    heading[i] = _get_heading(norm_quaternion[0], norm_quaternion[1],norm_quaternion[2],norm_quaternion[3])



In [ ]:
plt.hist(heading)

In [ ]:
def plot_onix_time_orientation(onix_time, video_path, trans_data, processed_bno, heading, rvec, tvec, cameraMatrix, distCoeffs):
    #obtaining the closest frame
    frame_start = get_onix_frame(video_path, onix_time)
    #transforming coordinates
    proj_data = lighthouse2aruco(trans_data)
    #selecting index
    lighthouse_time = find_nearest(proj_data['clock'], onix_time)
    index = proj_data[proj_data['clock']==lighthouse_time].index[0]
    objectPoints = np.array([proj_data['x'][index], proj_data['y'][index], proj_data['z'][index]])
    objectPoints = np.transpose(objectPoints)
    #Obtaining heading
    heading_time = find_nearest(processed_bno['onix_time'], onix_time)
    heading_index = processed_bno['onix_time'].index[processed_bno['onix_time'].eq(heading_time)][0]
    current_heading = heading[heading_index]
    #Obtaining heading point
    heading_point_x = objectPoints[0]+100*math.cos(current_heading)
    heading_point_y = objectPoints[1]+100*math.sin(current_heading)
    heading_point = np.transpose(np.array([heading_point_x, heading_point_y, 0]))
    #transforming and plotting
    imagePoints, jacobian = cv2.projectPoints(objectPoints, rvec, tvec, cameraMatrix, distCoeffs)
    headingPoints, jacobian = cv2.projectPoints(heading_point, rvec, tvec, cameraMatrix, distCoeffs)
    coordinates, heading_coordinates = imagePoints[0][0], headingPoints[0][0]
    fig, axs = plt.subplots(1, 1, figsize=(50, 50))
    axs.imshow(frame_start)
    axs.scatter(coordinates[0], coordinates[1], marker='+', s=15e3)
    print(heading_coordinates)
    axs.plot([coordinates[0], heading_coordinates[0]], [coordinates[1], heading_coordinates[1]], lw=12, linestyle='solid', color = 'red')
    return fig

#plot_onix_time_orientation(onix_time, video_path, trans_data, rvec, tvec, cameraMatrix, distCoeffs)


In [ ]:
def _plot_onix_time_orientation(onix_time, video_path, avg_first_frame, proj_data, processed_bno, heading, rvec, tvec, cameraMatrix, distCoeffs, normalise=False):
    """
    Same as plot_onix_time, but for efficiency I took out the transformation step to use it in generating the videos, so it works directly with
    proj_data. 
    """

    #obtaining the closest frame
    frame_start = get_onix_frame(video_path, onix_time, avg_first_frame = avg_first_frame, normalise=normalise)
    #selecting index
    lighthouse_index = find_nearest(proj_data['clock'], onix_time)
    index = proj_data[proj_data['clock']==lighthouse_index].index[0]
    objectPoints = np.array([proj_data['x'][index], proj_data['y'][index], proj_data['z'][index]])
    objectPoints = np.transpose(objectPoints)
    objectPoints[2]==0
    #Obtaining heading
    heading_time = find_nearest(processed_bno['onix_time'], onix_time)
    heading_index = processed_bno['onix_time'].index[processed_bno['onix_time'].eq(heading_time)][0]
    if heading[heading_index]>0:
        current_heading = heading[heading_index]-(math.pi+0.236332)#The offset looks like pi and 12 degrees, by eye
    else:
        current_heading = heading[heading_index]+(math.pi-0.236332)
    #Obtaining heading point
    heading_point_x = objectPoints[0]+100*math.cos(current_heading)
    heading_point_y = objectPoints[1]+100*math.sin(current_heading)
    heading_point = np.transpose(np.array([heading_point_x, heading_point_y, 0]))
    #transforming and plotting
    imagePoints, jacobian = cv2.projectPoints(objectPoints, rvec, tvec, cameraMatrix, distCoeffs)
    headingPoints, jacobian = cv2.projectPoints(heading_point, rvec, tvec, cameraMatrix, distCoeffs)
    coordinates, heading_coordinates = imagePoints[0][0], headingPoints[0][0]
    fig, axs = plt.subplots(1, 1, figsize=(50, 50))
    axs.imshow(frame_start)
    axs.scatter(coordinates[0], coordinates[1], marker='+', s=15e3)
    axs.plot([coordinates[0], heading_coordinates[0]], [coordinates[1], heading_coordinates[1]], lw=12, linestyle='solid', color = 'red')
    return fig



In [ ]:
def plot_interval_orientation_s(start, end, video_path, trans_data, processed_bno, heading, rvec, tvec, cameraMatrix, distCoeffs, normalise=False):

    frame_times_s = np.arange(start, end, (1/15))

    # Define the output file name, codec, frame rate, and dimensions
    output_file = f'output_orient_video_{str(start)}_{str(end)}.mp4'
    codec = cv2.VideoWriter_fourcc(*'mp4v')  # Choose the appropriate codec
    frame_rate = 15  # Desired frame rate
    frame_width = 3600 #as measured from current matplotlib output, maybe better to change programatically to ensure robustness when changing
    #plotting function?
    frame_height = 3600
    avg_first_frame = get_avg_first_frame(video_path)

    # Create the video writer
    video_writer = cv2.VideoWriter(output_file, codec, frame_rate, (frame_width, frame_height))

    proj_data = lighthouse2aruco(trans_data)

    for i in frame_times_s:
        time_in_s = i
        onix_time = proj_data['clock'][0]+(time_in_s*250e6)
        fig = _plot_onix_time_orientation(onix_time, video_path, avg_first_frame, proj_data, processed_bno, heading, rvec, tvec, cameraMatrix, distCoeffs, normalise = normalise)
        image = plut.get_img_from_fig(fig)
        plt.close(fig)
        bgr_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # Write the frame to the video
        video_writer.write(bgr_image)

    video_writer.release()
    cv2.destroyAllWindows()

start = 30
end = 60

plot_interval_orientation_s(start, end, video_path, trans_data, processed_bno, heading, rvec, tvec, cameraMatrix, distCoeffs)

start = 1603
end = 1633

plot_interval_orientation_s(start, end, video_path, trans_data, processed_bno, heading, rvec, tvec, cameraMatrix, distCoeffs)

In [ ]:
clock_s = (trans_data['clock']-trans_data['clock'][0])/250e6
dif_clock = np.diff(clock_s)
plt.hist(dif_clock)
print(f'The median period is {np.median(dif_clock)}s, therefore the median frequency is {1/(np.median(dif_clock))}Hz')

In [ ]:
clock_s = (processed_bno['onix_time']-processed_bno['onix_time'][0])/250e6
dif_clock = np.diff(clock_s)
plt.hist(dif_clock)
print(f'The median period is {np.median(dif_clock)}s, therefore the median frequency is {1/(np.median(dif_clock))}Hz')